# BLUESKY

This notebook demos the webscraping and analysis abilities of BLUESKY.
BLUESKY is a Python package that returns information about vehicle prices.

In [6]:
# IMPORT STATEMENTS
from BLUESKY.scrapers.bs4_scraper import scrape_data_payload
from BLUESKY.scrapers.genurls import gen_cars_com_urls

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Generating Target URLS to scrape from

In [28]:
makes = ["ford", "toyota", "toyota", ]
models = ["mustang", "camry", "supra",]
# TODO the generator should probably be rewritten to take a dict for make and model
# {"ford": ["mustang"], "toyota": ["supra", "camry"], "acura": ["integra"], }
url_targets = gen_cars_com_urls(makes=makes,models=models)
#url_targets

In [9]:
temp = scrape_data_payload(url_targets)

Processing the scraped data

In [20]:
columns = ["make", "model", "model_year", "trim", "mileage", "price", "listing_id"]

df = pd.DataFrame(temp, columns=columns)

In [30]:
mustangs = df[
    (df["make"] == "ford")
    & (df["model"] == "mustang")
    & (df["trim"].str.contains("gt"))
]

supras = df[
    (df["make"] == "toyota")
    & (df["model"] == "supra")
    & (df["trim"].str.contains("3.0"))
]

camrys = df[
    (df["make"] == "toyota")
    & (df["model"] == "camry")
    & (df["trim"] == "se")
]


Beginning analysis. 

TODO:
- change model years to delta time from present
- depreciation per mile, per year
    - per mile per year?